# ReID

In [ ]:
"""
a basline for Person ReID
market1501 --> ask @ TA
training data: '00001': [img1, img2, img3,...]
testing data: [img ...]? 

Given a query photo, recognize person ID ?  < == image search / retrieval
Gallery : image database <== training data + testing data (only a few)

q, g_i in G : learn a (similarity / distance) function: s(q, g_i) ? 

Suppose q and g_i are feature vectors, s(q, g_i)  = q dot_product g_i 

ConvNet e.g. ResNet-50 / MobileNet  == > Train a ConvNet

"""

In [1]:
img_width = 64
img_height =64
learning_rate = 0.01

# pip install tensorflow
from tensorflow.keras.optimizers import SGD
optimizer = SGD(learning_rate = learning_rate )
batch_size = 128
nbr_epochs = 100
data_folder = '/Users/DanDan/Downloads/Market-1501-v15.09.15/bounding_box_train'
# 12936 images in the training set

In [3]:
import os
data_root = os.path.join(os.getcwd(), data_folder)
image_names = sorted([x for x in os.listdir(data_root) if x.endswith('.jpg')])
# image_name: '0002_c2s1_ooo451_003'

img_name, img_path = zip(*[(img_file_png[:-4], os.path.join(data_root, img_file_png)
                           ) for img_file_png in image_names])

person_id_original_list = [x[:4] for x in img_name]

nbr_person_ids = len(set(person_id_original_list))
print('number of person ids', nbr_person_ids)

number of person ids 751


In [9]:
from sklearn.preprocessing import LabelEncoder
id_encoder = LabelEncoder()
id_encoder.fit(person_id_original_list)
person_id_encoded = id_encoder.transform(person_id_original_list)

from sklearn.model_selection import train_test_split
train_img_path, val_img_path, train_person_ids, val_person_ids = train_test_split(
                                img_path,person_id_encoded, test_size=0.2, random_state= 42)
print('# train images: {}, # val images: {}, # image labels: {}'.format(
                len(train_img_path), len(val_img_path), len(set(train_person_ids))))

# train images: 10348, # val images: 2588, # image labels: 751


In [11]:
# input_shape: (batch, height, width, channels)
# load pretrained MobileNet

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
cnn_model = MobileNetV2(include_top = False, weights = 'imagenet', alpha=0.5, 
                       input_shape = (img_height, img_width, 3), pooling = 'max')

2021-11-05 13:57:32.723686: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


3211264/3201480 [==============================] - 0s 0us/step


In [13]:
global_pool = cnn_model.layers[-1].output

In [20]:
from tensorflow.keras.layers import Dense, Activation
dense = Dense(nbr_person_ids)(global_pool)
softmax_output=Activation('softmax')(dense)

from tensorflow.keras.models import Model
baseline_model = Model(cnn_model.input, softmax_output)
baseline_model.summary()

baseline_model.compile(loss = 'categorical_crossentropy',
                      optimizer = optimizer,
                      metrics = ['accuracy'])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 32, 32, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 32, 32, 16)   64          Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 32, 32, 16)   0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [21]:
import numpy as np
# data augmentation: https:github.com/aleju/imgaug
# pip install imgaug
import imgaug as ia
from imgaug import augmenters as iaa

seq = iaa.Sequential()


def load_img_batch(img_path_list, img_label_list, nbr_classes, img_width, img_height):
    batch_size = len(img_path_list)
    
    X_batch = np.zeros((batch_size, img_height, img_width, 3))
    Y_batch = np.zeros((batch_size, nbr_classes)) # label: one-hot encoding

    for i in range(batch_size):
        img_path = img_path_list[i]
        
        import cv2
        img_bgr = cv2.imread(img_path) # img.shape(128, 64, 3)
        if img_bgr.shape != (img_height, img_width, 3):
            img_bgr = cv2.resize(img_bgr, (img_width, img_height))
            
        img = img_bgr[:, :, :: -1]

        X_batch[i] = img
        
        if img_label_list is not None:
            label = img_label_list[i]
            Y_batch[i, label] = 1
            
    if img_label_list is not None:
        return X_batch, Y_batch
    else:
        return X_batch
    
def generator_batch(img_path_list, img_label_list, nbr_classes, img_width, img_height,
                    batch_size=32, shuffle=False, save_to_dir = None, augment=True): 
    # img_path_list : ['/home/data/1.jpg', ....]
    # img_label_list: [7, 23, 4, ...]
    # output: yield (X_batch, y_batch)  or X_batch 
    
    N = len(img_path_list)
    if shuffle:
        from sklearn.utils import shuffle as shuffle_tuple
        img_path_list, img_label_list = shuffle_tuple(img_path_list, img_label_list)
        
    batch_index=0
    while True:
        current_index = (batch_index * batch_size) % N
        if N >= (current_index + batch_size):
            current_batch_size = batch_size
            batch_index+=1
        else:
            current_batch_size = N - current_index
            batch_index =0
        X_batch, Y_batch = load_img_batch(
            img_path_list[current_index : current_index + current_batch_size],
            img_label_list[current_index: current_index+ current_batch_size],
            nbr_classes, img_width, img_height
        )
        
        if augment:
            X_batch = X_batch.astype(np.uint8)
            X_batch_aug = seq.augment_images(X_batch)
            X_batch = X_batch_aug
            
        X_batch = X_batch / 255.
        X_batch = (X_batch - np.array([0.485, 0.486, 0.406])) / np.array([0.229, 0.224, 0.225])
        yield(X_batch, Y_batch)

In [23]:
# Data Loading
# from utils import generator_batch()
train_generator = generator_batch(img_path_list = train_img_path, 
                                  img_label_list = train_person_ids,
                                 nbr_classes = nbr_person_ids,
                                 img_width=img_width, img_height= img_height,
                                 batch_size=batch_size, shuffle=True,
                                 save_to_dir = False, augment = True)
val_generator = generator_batch(img_path_list = val_img_path,
                               img_label_list = val_person_ids, nbr_classes=nbr_person_ids,
                               img_width=img_width, img_height= img_height,
                               batch_size = batch_size, shuffle = False,
                               save_to_dir = False, augment= False)

from tensorflow.keras.callbacks import Checkpoint
# 回调函数
checkpoint = ModelCheckpoint('./cnn_baseline.h5', monitor='val_accuracy',
                            verbose=1, save_best_only=True)

baseline_model.fit(train_generator, 
                   steps_per_epoch=len(train_img_path) // batch_size,
                   validation_data = val_generator,
                  validation_steps = len(val_img_path) //batch_size,
                  batch_size=batch_size,verbose=1,shuffle=True,
                  epochs=nbr_epochs,callbacks=[checkpoint],
                  
                  )

ImportError: cannot import name 'Checkpoint' from 'tensorflow.keras.callbacks' (/opt/anaconda3/lib/python3.8/site-packages/keras/api/_v2/keras/callbacks/__init__.py)